In [1]:
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd #tratamiento de datos
import numpy as np #operaciones matriciales y con vectores
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
# NLTK Stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('spanish')
import spacy
nlp = spacy.load('es_core_news_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/garzuzo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Importamos los modelos generados anteriormente

In [2]:
lda_model=joblib.load('modelo_entrenado.pkl')

In [3]:
vectorizer = joblib.load('modelo_vectorizer.pkl')

In [4]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=30)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,...,Word 20,Word 21,Word 22,Word 23,Word 24,Word 25,Word 26,Word 27,Word 28,Word 29
Topic 0,igualdad,amigos,entorno,cuidar,espacios,bienestar,acciones,seguro,construccion,calle,...,territorios,comparto,soledad,alcanzar,relaciones,aportar,salud,diversion,situaciones,participacion
Topic 1,respeto,personas,demas_personas,diferencias,contar,respeto_hacia,tratar,raza,gente,ambiente,...,ser_tolerante,poder_convivir,medio_ambiente,cero,tierra,barrios,hija,corrupcion,ciudadano,hallar
Topic 2,tranquilo,sociedad,no_violencia,vida,mejor,educacion,oportunidades,social,saber,siempre,...,necesitar,conocer,dejar,ausencia,diferencia,aceptacion,sentirme,siento,convivencia,no_discriminacion
Topic 3,armonia,felicidad,generar,hijos,corazon,vida,sana_convivencia,alegria,ser_humano,barrio,...,transmitir,dar,reflejo,familias,unir,brindar,encontrar,considerar,poder_compartir,futuro
Topic 4,tranquilidad,respeto,solidaridad,confianza,personal,tolerancia,reflejar,valores,hogar,dialogo,...,bien,educacion,permitir,caminar,comunitario,equidad,diferencia,diversidad,economica,derechos_sociales
Topic 5,construir,mundo,libertad,respeto,armonia,lograr,perdonar,bueno,derechos,territorio,...,deporte,aceptar,ayudar,cumplir,partir,practicar,necesario,fundamental,capacidad,cultura_ciudadana
Topic 6,amor,respeto,tolerancia,pensar,conflictos,personas,seguridad,diferente,persona,respeto_tolerancia,...,mas_importante,aceptar,pueblo,convivir,mediar,alrededor,positivo,importante,social,servir
Topic 7,dios,solo,entender,existir,musica,disfrutar,escuchar,projimo,conflicto,encuentro,...,situacion,comprender,amistad,hablar,expresar,serenidad,darle,comer,gustar,madre
Topic 8,naturaleza,casa,sentir,convivencia,problemas,representar,llegar,conciencia,forma,tener_buena,...,animales,llego,parque,paloma,seres_queridos,significar,valores,caminar,honestidad,contaminacion
Topic 9,familia,vivir,comunidad,personas,armonia,compartir,bien,interior,union,bueno,...,crecer,serio,perdon,tipo,familiar,entendimiento,armas,dibujar,ninos,problema


## Procesamiento de un nuevo texto

Evaluaremos un nuevo texto, pero antes tenemos que hacer una respectiva limpieza.

In [5]:
text= "Para mi la paz es tener tranquilidad en mi hogar con mi familia, mis amigas y mi perro"

In [6]:
text= re.sub('[,\.\'\"!\)(?0-9]', '', text).lower().strip()

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc= True))  # deacc=True removes punctuations

data = [text]
data_tokenized = list(sent_to_words(data)) 

In [8]:

stop_words_exceptions=["no"]
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if (word not in stop_words) or (word in stop_words_exceptions)] for doc in texts]
    
    
data_tokenized_nostops= remove_stopwords(data_tokenized)

In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_tokenized, min_count=7, threshold=5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_tokenized], threshold=5)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

data_bigrams_nonstops=make_bigrams(data_tokenized_nostops)

In [10]:
data_bigrams_nonstops

[['paz', 'tener', 'tranquilidad', 'hogar', 'familia', 'amigas', 'perro']]

In [11]:
data = data_bigrams_nonstops

data_prepared=[]
for row in data:
    text=""
    for word in row:
        if(len(word) > 3):
            text+=word + " "
    data_prepared.append(text.rstrip())

print(data_prepared)

['tener tranquilidad hogar familia amigas perro']


In [12]:
allowed_postags=['ADJ', 'VERB', 'ADV']
stopwordsToken=["demas","tener","mismo", "poder","cada","tambien", "hacer"]
unifiedWord={"respetar":"respeto", "violencia":"no_violencia","musicar":"musica","guerra":"no_guerra"}
data_list=[]

token_list=""
text=nlp(data_prepared[0])
for token in text:
    if (token.pos_ in allowed_postags) and (token.lemma_ not in stopwordsToken) and (token.text not in stopwordsToken):
        if token.lemma_ in unifiedWord:
            token_list+=unifiedWord[token.lemma_]+" "
        else :
            token_list+=token.lemma_+" "

    elif (token.is_stop is not True) and (token.lemma_ not in stopwordsToken) and (token.text not in stopwordsToken):
        if token.text in unifiedWord:
            token_list+=unifiedWord[token.text]+" "
        else :
            token_list+=token.text+" "
data_list.append(token_list.rstrip())

In [13]:
data_list

['tranquilidad hogar familia amigas perro']

Ahora transformamos el texto procesado en una matriz de dispersión

In [14]:
text_processed=vectorizer.transform(data_list)
print(text_processed)

  (0, 181)	1
  (0, 214)	1
  (0, 426)	1


In [15]:
topic_probability_scores = lda_model.transform(text_processed)

In [16]:
print(topic_probability_scores)

[[0.0250002  0.025      0.02500212 0.02500561 0.51681747 0.025
  0.02500464 0.025      0.02500153 0.28316842]]


In [17]:
np.argmax(topic_probability_scores)

4

In [18]:
topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()

In [19]:
print(', '.join(topic))

tranquilidad, respeto, solidaridad, confianza, personal, tolerancia, reflejar, valores, hogar, dialogo, construccion, unidad, medio_ambiente, adultos, generar, comun, interpersonal, practicar, normas, seres_humanos, bien, educacion, permitir, caminar, comunitario, equidad, diferencia, diversidad, economica, derechos_sociales
